In [1]:
# load models
# compare how much divergence between model 1 trained and original model for all

In [2]:

%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
from os.path import join as oj
import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
sys.path.append('../../acd')
sys.path.append('../../acd/visualization')
sys.path.append('../../acd/acd/util')
sys.path.append('../../acd/acd/scores')
sys.path.append('../../acd/acd/agglomeration')

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
import pandas as pd
from os.path import join
import torch
import torch
import numpy as np
import seaborn as sns
from copy import deepcopy
from model import LSTMSentiment
import matplotlib.pyplot as plt
from os.path import isdir
import viz_1d as viz
import tiling_1d as tiling
import agg_1d as agg
import cd
import score_funcs
import dsets
from dsets.sst import dset
from dsets.sst.model import LSTMSentiment
# check out how two models differ
import torch.optim as O
import torch.nn as nn
from torchtext import data
from torchtext import datasets
from tqdm import tqdm_notebook, tqdm 
import pickle

In [3]:
sst_pkl = pickle.load(open('../../acd/dsets/sst/sst.pkl', 'rb'))
np.random.seed(42)

In [4]:
sst_pkl = pickle.load(open('../../acd/dsets/sst/sst.pkl', 'rb'))
np.random.seed(42)
vector_cache = os.path.join(os.getcwd(), '../data/.vector_cache/input_vectors.pt')
word_vectors ='glove.6B.300d'
batch_size=  50

In [5]:
trained_model_folder = '../models/trained_models'
init_model_folder = '../models/init_models'
trained_list = os.listdir(trained_model_folder)

In [6]:
init_models = os.listdir(init_model_folder)
init_model =torch.load(join(init_model_folder, init_models[0]))
init_comp_model =torch.load(join(init_model_folder, init_models[1]))

/home/lauri/.conda/envs/gpu_usage/lib/python3.6/site-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'model.LSTMSentiment' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [7]:

fnames = sorted([oj(trained_model_folder, fname) for fname in os.listdir(trained_model_folder)]) # filenames in the directory
results_list = [pd.Series(pkl.load(open(fname, "rb"))) for fname in tqdm_notebook(fnames) ] 
results = pd.concat(results_list, axis=1).T.infer_objects() # pandas dataframe w/ hyperparams and weights stored


In [8]:
# load test set

inputs = data.Field(lower= True)
answers = data.Field(sequential=False, unk_token=None)

train, dev, test = datasets.SST.splits(inputs, answers, fine_grained=False, train_subtrees=True,
                                       filter_pred=lambda ex: ex.label != 'neutral')

inputs.build_vocab(train, dev, test)

In [9]:
if word_vectors:
    if os.path.isfile(vector_cache):
        inputs.vocab.vectors = torch.load(vector_cache)
    else:
        inputs.vocab.load_vectors(word_vectors)
        makedirs(os.path.dirname(vector_cache))
        torch.save(inputs.vocab.vectors,vector_cache)
answers.build_vocab(train)
train_iter, dev_iter, test_iter = data.BucketIterator.splits(
    (train, dev, test), batch_size=batch_size, device=torch.device(0))

In [10]:
trained_model = deepcopy(init_model)
trained_model.load_state_dict(results['model_weights'][0])

In [14]:
print(calc_expl_divergence(init_model, init_comp_model, dev)) #divergence between the two original models

0.08425198558273665


In [15]:
divergence_comp = []
trained_model = deepcopy(init_model)
for i in tqdm_notebook(range(len(results))):
    trained_model.load_state_dict(results['comp_model_weights'][i])
    divergence_comp.append(calc_expl_divergence(trained_model, init_model, dev))
results['divergence_comp'] = divergence_comp    

/home/lauri/.conda/envs/gpu_usage/lib/python3.6/site-packages/torch/nn/modules/rnn.py:179: RuntimeWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
  self.dropout, self.training, self.bidirectional, self.batch_first)


In [16]:
divergence = []
trained_model = deepcopy(init_model)
for i in tqdm_notebook(range(len(results))):
    trained_model.load_state_dict(results['model_weights'][i])
    divergence.append(calc_expl_divergence(trained_model, init_comp_model, dev))
results['divergence'] = divergence    

In [17]:
results['divergence'] = divergence
results["final_acc"] = [x[-1] for x in results["accs_test"]]
results["final_cd"] = [x[-1] for x in results["explanation_divergence"]]
results[['train_both','sparse_signal', 'signal_strength', 'final_acc', 'final_cd', 'divergence','divergence_comp']].sort_values(by=['divergence'])

,train_both,sparse_signal,signal_strength,final_acc,final_cd,divergence,divergence_comp
9,False,True,10.0,81.0,203.205975,0.055893,0.079461
7,False,False,10.0,82.0,261.337331,0.059441,0.084055
12,False,True,10.0,81.0,226.873438,0.064658,0.089416
10,False,False,1.0,80.0,274.506503,0.065037,0.080025
3,False,True,1.0,79.0,212.790776,0.071519,0.086711
0,False,True,1.0,79.0,245.139168,0.071601,0.086350
5,True,True,10.0,79.0,5.554571,0.176107,0.199994
6,True,True,10.0,80.0,5.093287,0.205032,0.220881
4,True,False,10.0,77.0,5.000549,0.244531,0.272252
2,True,True,1.0,80.0,9.895252,0.251249,0.307977


In [11]:
def kl_divergence(in1, in2):
    a= (in1-in2)
    b = (torch.log(in1) - torch.log(in2))

    return (a*b).sum(dim=1).cpu().detach().numpy()
def softmax(scores):
    return torch.nn.functional.softmax(torch.stack((scores[0].view(-1),scores[1].view(-1)), 1), dim = 1)
criterion = nn.CrossEntropyLoss()
def calc_expl_divergence(model1, model2, dataset):
    ''' calculate explanation divergence between two models on the given dataset. Return'''
    len_data = len(dev)
    
    expl_penalty = np.zeros((len_data))
    is_correct = np.zeros((2, len_data))
    num_reps = 5

    model1.eval()
    model2.eval()
    with torch.no_grad():

        cur_idx = 0
        for dev_batch_idx, dev_batch in enumerate(dev_iter):
            
            batch_length, batch_num = dev_batch.text.shape
            answer1 = model1(dev_batch)
            answer2 = model2(dev_batch)
            
            for i in range(num_reps):
                start = np.random.randint(batch_length-1)
                stop = start + np.random.randint(batch_length-start)
                expl_penalty[cur_idx:cur_idx+batch_num] +=(cd.cd_penalty(dev_batch, model1, model2, start, stop)).cpu().numpy()
                
            is_correct[0, cur_idx:cur_idx+batch_num]=(((torch.max(answer1, 1)[1].view(dev_batch.label.size()).data == dev_batch.label.data))).cpu().numpy()
            is_correct[1, cur_idx:cur_idx+batch_num]=(((torch.max(answer2, 1)[1].view(dev_batch.label.size()).data == dev_batch.label.data))).cpu().numpy()
            cur_idx+=batch_num
        expl_penalty /=num_reps
        one_wrong_idx = np.where(is_correct.mean(axis=0) ==0.5)[0]
        both_correct_idx =np.where(is_correct.mean(axis=0) ==1)[0]
        both_wrong_idx =np.where(is_correct.mean(axis=0) ==0)[0]
        return expl_penalty.mean()#, expl_div[both_correct_idx].mean(), expl_div[one_wrong_idx].mean(), expl_div[both_wrong_idx].mean()


In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# form class to hold data
class B:
    text = torch.zeros(1).to(device)

In [12]:
# base parameters
sweep_dim = 1 # how large chunks of text should be considered (1 for words)
method = 'cd' # build_up, break_down, cd
percentile_include = 99.5 # keep this very high so we don't add too many words at once
num_iters = 25 # maximum number of iterations (rarely reached)

# text and label
sentence = ['a', 'great', 'ensemble', 'cast', 'ca', 'n\'t', 'lift', 'this', 'heartfelt', 'enterprise', 'out', 'of', 'the', 'familiar', '.'] # note this is a real example from the dataset
# sentence = ['not', 'good', ',', 'not', 'bad', ',', 'just', 'okay'] # any text with words from this dataset can be interpreted
label = 1 # 0 if positive 1 if negative


def batch_from_str_list(s):
    batch = B()
    nums = np.expand_dims(np.array([sst_pkl['stoi'][x] for x in s]).transpose(), axis=1)
    batch.text = torch.LongTensor(nums).to(device) #cuda()
    return batch

# prepare inputs
batch = batch_from_str_list(sentence)
scores_all = init_model(batch).data.cpu().numpy()[0] # predict
label_pred = np.argmax(scores_all) # get predicted class

# agglomerate
lists = agg.agglomerate(init_model, batch, percentile_include, method, sweep_dim, # only works for sweep_dim = 1
                    label_pred, num_iters=num_iters, device=device) # see agg_1d.agglomerate to understand what this dictionary contains
lists = agg.collapse_tree(lists) # don't show redundant joins

# visualize
viz.word_heatmap(sentence, lists, label_pred, label, fontsize=9)

NameError: name 'B' is not defined